## 5.11 Klasyfikacja za pomocą SVM
Spróbujmy, tym razem za pomocą SVM, rozwiązać kolejny raz ten sam problem. Jest to dość typowe zjawisko w przypadku tworzenia rozwiązań ML - należy stworzyć kilka modeli, aby później wybrać ten najlepszy, względnie wystarczająco dobry, ale wydajny. Poprzednio testowaliśmy KNN i jego wydajność dość mocno zależy od tego na jak wielu wektorach trenowaliśmy nasz model, co czyni go mało użytecznym gdy danych jest naprawdę sporo.

In [1]:
import pandas as pd

In [2]:
credit_cards_df = pd.read_parquet("../data/credit-cards-reduced.parquet")
credit_cards_df.T

ID,1,2,3,4,5,6,7,8,9,10,...,29991,29992,29993,29994,29995,29996,29997,29998,29999,30000
LIMIT_BAL,20000,120000,90000,50000,50000,50000,500000,100000,140000,20000,...,140000,210000,10000,100000,80000,220000,150000,30000,80000,50000
AGE,24,26,34,37,57,37,29,23,28,35,...,41,34,43,38,34,39,43,37,41,46
PAY_1,2,-1,0,0,-1,0,0,0,0,-2,...,0,3,0,0,2,0,-1,4,1,0
PAY_2,2,2,0,0,0,0,0,-1,0,-2,...,0,2,0,-1,2,0,-1,3,-1,0
PAY_3,-1,0,0,0,-1,0,0,-1,2,-2,...,0,2,0,-1,2,0,-1,2,0,0
PAY_4,-1,0,0,0,0,0,0,0,0,-2,...,0,2,-2,0,2,0,-1,-1,0,0
PAY_5,-2,0,0,0,0,0,0,0,0,-1,...,0,2,-2,0,2,0,0,0,0,0
PAY_6,-2,2,0,0,0,0,0,-1,0,-1,...,0,2,-2,0,2,0,0,0,-1,0
BILL_AMT1,3913,2682,29239,46990,8617,64400,367965,11876,11285,0,...,138325,2500,8802,3042,72557,188948,1683,3565,-1645,47929
BILL_AMT2,3102,1725,14027,48233,5670,57069,412023,380,14096,0,...,137142,2500,10400,1427,77708,192815,1828,3356,78379,48905


Celowo nie ładujemy pełnego zbioru, żeby  być w stanie porównać ze sobą wszystkie stworzone dotychczas rozwiązania. Regresja logistyczna działała najlepiej właśnie na tym podzbiorze i niekoniecznie oznacza to, że inne metody nie będą w stanie wyciągnąć żadnych wniosków z pozostałych atrybutów. Wybór najlepszego zestawu cech jest dość specyficzny dla każdego modelu z osobna - to że w naszym przypadku regresja logistyczna nie dała rady z pewnych informacji skorzystać, nie może być podstawą do stwierdzenia, że nie mają one wartości.

In [3]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test = train_test_split(credit_cards_df, test_size=0.2, random_state=2020)

In [6]:
svm = LinearSVC()

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [8]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", LinearSVC())
])

In [9]:
pipeline.fit(credit_cards_df.drop(columns="DEFAULT"), credit_cards_df["DEFAULT"])

Pipeline(steps=[('scaler', StandardScaler()), ('classifier', LinearSVC())])

In [10]:
from sklearn.metrics import f1_score

In [11]:
f1_score(X_test["DEFAULT"], pipeline.predict(X_test.drop(columns="DEFAULT")))

np.float64(0.4239917269906929)

In [12]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", LinearSVC(max_iter=10000))
])
pipeline.fit(X_train.drop(columns="DEFAULT"), X_train["DEFAULT"])

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LinearSVC(max_iter=10000))])

In [14]:
f1_score(X_test["DEFAULT"], pipeline.predict(X_test.drop(columns="DEFAULT")))

np.float64(0.41935483870967744)